# Compound

> Fill in a module description here

In [ ]:
#| default_exp compound.core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from exex.core import *

In [ ]:
#| export
class Matter:
    pass

All properties that a compound has always being governed by some laws.

In [ ]:
#| export
class Compound(Matter):
    def __init__(self):
        self._properties = dict()
        self._laws = []
    
    def add_laws(self, laws):
        for law in laws:
            self._laws.append(law(compound=self))

In [ ]:
H2O = Compound()

In [ ]:
#H2O._properties.append(Mass)